In [1]:
"""
IS2_S2_cryo2ice_search.ipynb
Alek Petty, adapted from an earlier M. Bagnardi script.

Deescription:
- This notebook searches for semi-coincident ICESat-2 and Sentinel-2 data.
- The code finds Sentinel-2 images with ICESat-2 data falling within the image's footprint.
- The code was updated to read in ATl07 data from the new cloud storage using earthdata, see relevant tutorials:
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/ATL06-direct-access_rendered.ipynb
     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/nsidc_daac_uwg_cloud_access_tutorial_rendered.ipynb (No longer hosted!!)
- Include new filtering to only look for images where we have cryo2ice (CryoSat-2 coincident) data based on external file. 
 
To do:
- Integrate the GEE or other cloud-based S-2 catalog. What data is on those comapred to the sentinelhub?
- Check all beams iunstead of just the middle beam.
- Add in other imagery/satellite sensors.

Notes:
- Another way of doing this is to use the RGT (reference ground track) data from ICESat-2.
    This could help if you want to look at future data I guess, but downside is it doesn't say where the data actually is on yhe ground, 
    just theoretical tracks.
- Uses the middle strong beam but could be easily adapted to check across all beams.=
- Tested with the adapted 'geo' conda env on cryocloud if you want to convert into a python script.

Output:

Update history:
 - Current notebook created in Apr 2024.

"""

"\nIS2_S2_cryo2ice_search.ipynb\nAlek Petty, adapted from an earlier M. Bagnardi script.\n\nDeescription:\n- This notebook searches for semi-coincident ICESat-2 and Sentinel-2 data.\n- The code finds Sentinel-2 images with ICESat-2 data falling within the image's footprint.\n- The code was updayed to read in ATl07 data from the new cloud storage using earthdata following this earthdata tutorial:\n     https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/ICESat-2_Cloud_Access/nsidc_daac_uwg_cloud_access_tutorial_rendered.ipynb\n- Include new filtering to only look for images where we have cryo2ice (CryoSat-2 coincident) data based on external file. \n \nTo do:\n- Integrate the GEE or other cloud-based S-2 catalog. What data is on those comapred to the sentinelhub?\n- Check all beams iunstead of just the middle beam.\n- Add in other imagery/satellite sensors.\n\nNotes:\n- Another way of doing this is to use the RGT (reference ground track) data from ICESat-2.\n    This could

In [2]:
# Uncomment these out when running for the first time on cryocloud
#%pip install sentinelsat
#%pip install earthdata
#%pip install astropy

In [3]:
#from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import datetime as dt
import h5py
import pandas as pd
# Need to do this for geopandas for some reason
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import glob
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os
from astropy.time import Time
import numpy as np
from PIL import Image
import io
import requests
import time
import earthaccess
import xarray as xr

In [24]:
%pip install openeo
import openeo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.6/253.6 kB 3.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Sentinel-hub stuff

# https://openeo.dataspace.copernicus.eu
# - has a nice summary of the S-2 datasets

#sentinel2_product='S2MSI1C'
#deltatime=20.0
#maxcloud=10
#lower_lat=60

#username = 'akpetty'
#password = 'Icebridge01!'

#sentinelsat.exceptions.UnauthorizedError: Invalid user name or password. Note that account creation and password changes may take up to a week to propagate to the 'https://apihub.copernicus.eu/apihub/' API URL you are using. Consider switching to 'https://scihub.copernicus.eu/dhus/' instead in the mean time.
# Try https://apihub.copernicus.eu/apihub (Marco) or https://scihub.copernicus.eu/dhus/
# Initialize access to API
#api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus/')


In [25]:

# Connect to openEO back-end.
connection = openeo.connect("openeo.vito.be").authenticate_oidc()

connection.list_collection_ids()


#connection.load_collection(
#    "SENTINEL2_L2A",
#    ...,
#    max_cloud_cover=80,
#)

Visit https://aai.egi.eu/device?user_code=LPIR-DOHY 📋 to authenticate.

[#########----------------------------] ✅ Authorized successfully

Authenticated using device code flow.


['MAPEO_WATER_TUR_V1',
 'COP_DEM_EU_25M',
 'ESA_WORLDCEREAL_ACTIVECROPLAND',
 'ESA_WORLDCEREAL_IRRIGATION',
 'ESA_WORLDCEREAL_TEMPORARYCROPS',
 'ESA_WORLDCEREAL_WINTERCEREALS',
 'ESA_WORLDCEREAL_MAIZE',
 'ESA_WORLDCEREAL_SPRINGCEREALS',
 'CGLS_GDMP300_V1_GLOBAL',
 'CGLS_GDMP_V2_GLOBAL',
 'SENTINEL1_GRD_SIGMA0',
 'S1_GRD_SIGMA0_ASCENDING',
 'S1_GRD_SIGMA0_DESCENDING',
 'SENTINEL3_SYNERGY_VG1',
 'SENTINEL3_SYNERGY_VG10',
 'TERRASCOPE_S2_FAPAR_V2',
 'TERRASCOPE_S2_NDVI_V2',
 'TERRASCOPE_S2_LAI_V2',
 'TERRASCOPE_S2_FCOVER_V2',
 'TERRASCOPE_S2_TOC_V2',
 'TERRASCOPE_S1_SLC_COHERENCE_V1',
 'SENTINEL1_GAMMA0_SENTINELHUB',
 'SENTINEL1_GRD',
 'SENTINEL2_L1C_SENTINELHUB',
 'SENTINEL2_L2A_SENTINELHUB',
 'SENTINEL2_L2A_MOSAIC_120',
 'PROBAV_L3_S10_TOC_333M',
 'PROBAV_L3_S5_TOC_100M',
 'PROBAV_L3_S1_TOC_100M',
 'PROBAV_L3_S1_TOC_333M',
 'TERRASCOPE_S5P_L3_NO2_TD_V1',
 'TERRASCOPE_S5P_L3_NO2_TM_V1',
 'TERRASCOPE_S5P_L3_NO2_TY_V1',
 'TERRASCOPE_S5P_L3_CO_TD_V1',
 'TERRASCOPE_S5P_L3_CO_TM_V1',
 'TERRAS

In [10]:
# ICESat-2/earthaccess stuff
auth = earthaccess.login()
granules_cloud = earthaccess.search_data(
    short_name = 'ATL07',
    version = '006',
    cloud_hosted = True,
    bounding_box = (-180,30,180,89),
    temporal = ("2021-03-01", "2021-03-02"),
)


Granules found: 14


In [20]:
# Open files (not found a way of not needing to do this..)
atl07_files = earthaccess.open(granules_cloud)

Opening 14 granules, approx size: 4.05 GB
using provider: NSIDC_CPRD


QUEUEING TASKS | :   0%|          | 0/14 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/14 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/14 [00:00<?, ?it/s]

In [26]:
# Loop over ICESat-2 granules
for atl07_file in atl07_files:
      
    # Check spacecraft orientation and assign string beam IDs
    ATL = h5py.File(atl07_file, 'r')
    orientation = ATL['/orbit_info/sc_orient'][0]

    # Only use central strong beam locations
    if orientation == 0:
        beamID = 'gt2l'
    elif orientation == 1:
        beamID = 'gt2r'
    else:
        print('Spacecraft orientation not found.')

    # Extract data info from granule
    ATL_start_time = ATL['/ancillary_data/data_start_utc'][0]
    ATL_start = pd.to_datetime(ATL_start_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')
    ATL_end_time = ATL['/ancillary_data/data_end_utc'][0]
    ATL_end = pd.to_datetime(ATL_end_time[:-8].decode('utf-8'), format='%Y-%m-%dT%H:%M:%S')

    GPS_epoch = ATL['ancillary_data/atlas_sdp_gps_epoch'][:]

    # Build dataframe with location of data
    ATL_dF = pd.DataFrame({'Longitude': ATL[beamID + '/sea_ice_segments/longitude'][::200],
                           'Latitude': ATL[beamID + '/sea_ice_segments/latitude'][::200],
                           })
    ATL_dF['coords'] = list(zip(ATL_dF['Longitude'], ATL_dF['Latitude']))
    ATL_dF['coords'] = ATL_dF['coords'].apply(Point)

    GPS_time = ATL[beamID + '/sea_ice_segments/delta_time'][::200] + GPS_epoch
    
    # Use astropy to convert from gps time to datetime
    ATL_tgps = Time(GPS_time, format='gps')
    ATL_utc = ATL_tgps.utc.iso
    ATL_dF['UTC'] = ATL_utc

    ATL_gfd = gpd.GeoDataFrame(ATL_dF, geometry='coords')
    ATL_gfd = ATL_gfd.set_crs(4326, allow_override=True)

    #try:
    # Search for Sentinel-2 coincident data

    S2_query = connection.load_collection(
        "SENTINEL2_L1C",
        temporal_extent=[ATL_start - dt.timedelta(minutes=deltatime), ATL_end + dt.timedelta(minutes=deltatime)],
        #bands=["B04", "B03", "B02"],
        max_cloud_cover=maxcloud,
    )

    S2_gdf = api.to_geodataframe(S2_query)

    S2_gdf_subset = S2_gdf[(S2_gdf.bounds.miny > lower_lat)]

    points_in_poly = gpd.tools.sjoin(ATL_gfd, S2_gdf_subset)
    #print(points_in_poly)
    
    try:
        # Empty geodataframes threw an exception here
        print('Number of overlapping tiles:', len(points_in_poly['title'].unique()))
    except:
        continue

    if len(points_in_poly['title'].unique()) > 0:

        # Filter products based on the tile ID
        #filtered_products = {k: v for k, v in products.items() if v['tileid'] == tile_id}

        print('all data:', S2_query)
        print('subset:', S2_gdf_subset)

        #print(S2_gdf_subset.title)

        # download all results from the search
        #api.download_all(S2_gdf_subset)

        #cwd = os.getcwd()
        #print('cwd:', cwd)

        atl_name = os.path.basename(ATL_filename)[:-3]
        filename = 'S2pairs_'+atl_name
        
        save_path = '/Users/aapetty/GitHub-output/ICESat-2-sea-ice-tools/IS2_S2_pair_Arctic/'+sentinel2_product+'/maxcloud'+str(maxcloud)+'_deltatime'+str(int(deltatime))+'/'+filename
        
        print('save_path:', save_path)
        
        if not os.path.exists(cwd+'/'+sentinel2_product):
             os.mkdir(cwd+'/'+sentinel2_product)

        if not os.path.exists(cwd+'/'+sentinel2_product+'/maxcloud'+str(maxcloud)+'_deltatime'+str(int(deltatime))):
             os.mkdir(cwd+'/'+sentinel2_product+'/maxcloud'+str(maxcloud)+'_deltatime'+str(int(deltatime)))

        if not os.path.exists(save_path):
             os.mkdir(save_path)

        
        f = open(save_path + "/" + filename+".txt", 'a')

        fig, ax = plt.subplots(1, 1)

        color = iter(plt.cm.rainbow(np.linspace(0, 1, len(points_in_poly['title'].unique()))))
        y_shift = 0.8

        for granule in points_in_poly['title'].unique():
            print(granule)
            c = next(color)
            y_shift = y_shift -0.05

            S2_gdf_subset[S2_gdf_subset['title'] == granule].boundary.plot(ax=ax, color=c)
            plt.text(1.2, y_shift, str(granule), transform=ax.transAxes, color=c)

        points_in_poly.plot(ax=ax, markersize=3, color='k', marker='o')
        plt.title(os.path.basename(ATL_filename))
        plt.xlabel('Longitude (deg.)')
        plt.ylabel('Latitude (deg.)')

        plt.text(1.2, y_shift -0.10, points_in_poly.iloc[0].UTC, transform=ax.transAxes, color='k')
        plt.text(1.2, y_shift -0.15, points_in_poly.iloc[-1].UTC, transform=ax.transAxes, color='k')

        plt.savefig(save_path + "/" + filename+".png", bbox_inches='tight', dpi=100)

        f.write(save_path + '\n')
        f.write('\n')
        f.write('Lat min: ' + str(min(points_in_poly.Latitude)) + '\n')
        f.write('Lat max: ' + str(max(points_in_poly.Latitude)) + '\n')
        f.write('Time start: ' + points_in_poly.iloc[0].UTC + '\n')
        f.write('Time end: ' + points_in_poly.iloc[-1].UTC + '\n')
        f.write('\n')
        f.write(str(points_in_poly['title'].unique()) + '\n')
        f.write('\n')
        f.write(str(points_in_poly['summary'].unique()) + '\n')
        f.write('\n')
        f.write(str(points_in_poly['link'].unique()) + '\n')
        f.write('\n')
        f.write(str(points_in_poly['link_icon'].unique()) + '\n')
        f.write('\n')

        f.close()

TypeError: object of type 'DataCube' has no len()

In [27]:
S2_query = connection.load_collection(
        "SENTINEL2_L1C",
        temporal_extent=[ATL_start - dt.timedelta(minutes=deltatime), ATL_end + dt.timedelta(minutes=deltatime)],
        #bands=["B04", "B03", "B02"],
        max_cloud_cover=maxcloud,
    )

In [34]:
S2_query.load()

AttributeError: 'DataCube' object has no attribute 'load'

In [28]:
B04 = S2_query.band("B04")

In [32]:
ATL_start - dt.timedelta(minutes=deltatime)

Timestamp('2021-03-01 00:57:58')

In [35]:
# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
api.to_geodataframe(S2_query)

TypeError: object of type 'DataCube' has no len()